# Libraries

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
import random
import os
import PIL
from contextlib import redirect_stdout
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import EarlyStopping

# Parameters

In [2]:
# Image parameters
img_width = 120
img_height = 120
img_channels = 3

# Preprocess data to train

In [3]:
# read file
data = pd.read_csv("../datasets/train.csv")

# replace labels {0: "cat", 1: "dog"}
data["label"] = data["label"].replace({0: "cat", 1: "dog"})

# train validation split
df_train, df_validation = train_test_split(data, test_size=0.15, random_state=2)

# reset train and validation dataframes indexes
df_train = df_train.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

# get some metadada from train and validation dataframes
df_train_shape = df_train.shape
df_validation_shape = df_validation.shape

In [4]:
df_train.head(2)

,image address,label
0,../datasets/train\cat.10290.jpg,cat
1,../datasets/train\dog.4021.jpg,dog


# Train and validation data

### Parameters

In [5]:
batch_size = 10

### Build data generators

In [6]:
train_data_generator = ImageDataGenerator(
    rotation_range=10,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_data_generator.flow_from_dataframe(
    df_train,
    x_col="image address",
    y_col="label",
    target_size=(img_height, img_width),
    class_mode="categorical",
    batch_size=batch_size
)

Found 21250 validated image filenames belonging to 2 classes.


In [7]:
validation_data_generator = ImageDataGenerator(rescale=1./255)

validation_generator = validation_data_generator.flow_from_dataframe(
    df_validation,
    x_col="image address",
    y_col="label",
    target_size=(img_height, img_width),
    class_mode="categorical",
    batch_size=batch_size
)

Found 3750 validated image filenames belonging to 2 classes.


# Model

### Create model archtecture

In [11]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation="relu", input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (4,4), activation="relu", input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(2, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Model summary

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 128)       65664     
                                                                 
 batch_normalization_1 (Batc  (None, 56, 56, 128)      5

### Define callbacks

In [15]:
earlystop = EarlyStopping(patience = 5)
callbacks = [earlystop]

### Train model

In [19]:
epochs = 10

history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=df_validation_shape[0]//batch_size,
    steps_per_epoch=df_train_shape[0]//batch_size,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/10
2125/2125 [==============================] - 634s 298ms/step - loss: 0.6406 - accuracy: 0.6587 - val_loss: 0.6290 - val_accuracy: 0.6547
Epoch 2/10
2125/2125 [==============================] - 430s 202ms/step - loss: 0.5256 - accuracy: 0.7421 - val_loss: 0.5455 - val_accuracy: 0.7637
Epoch 3/10
2125/2125 [==============================] - 431s 203ms/step - loss: 0.4921 - accuracy: 0.7646 - val_loss: 0.5663 - val_accuracy: 0.7069
Epoch 4/10
2125/2125 [==============================] - 430s 203ms/step - loss: 0.4737 - accuracy: 0.7773 - val_loss: 0.4990 - val_accuracy: 0.7976
Epoch 5/10
2125/2125 [==============================] - 430s 202ms/step - loss: 0.4590 - accuracy: 0.7881 - val_loss: 0.4235 - val_accuracy: 0.8176
Epoch 6/10
2125/2125 [==============================] - 430s 203ms/step - loss: 0.4362 - accuracy: 0.8024 - val_loss: 0.6682 - val_accuracy: 0.7824
Epoch 7/10
2125/2125 [==============================] - 438s 206ms/step - loss: 0.4245 - accuracy: 0.8076 - val_

### Save model

In [20]:
model.save("../backend/models/cats_vs_dogs_model_1.h5")

In [22]:
with open('../backend/models/cats_vs_dogs_model_1_summary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()